# Self-Organising Maps

Author: `Márcio Lopes Jr` 

*Master's student of `Computer Engineering, Intelligent Information Processing` at UFRN-Natal*.

## Libraries

In [1]:
# Data tools
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pylab

# Clustering
from minisom import MiniSom
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.metrics import davies_bouldin_score
from sklearn.manifold import TSNE
from dynamicTreeCut import cutreeHybrid
from scipy.spatial.distance import pdist

## Data Loading and Preprocessing

In [10]:
# Files subfolder
folder_name = 'files'

df = pd.read_csv(f"{folder_name}/A0.csv").set_index('cd_ibge')

In [11]:
# Removing outliers
lower, upper = df.ptb_rate.mean() - (df.ptb_rate.std()*3), df.ptb_rate.mean() + (df.ptb_rate.std()*3)
df = df[(df.ptb_rate > lower) & (df.ptb_rate < upper)]

# Normalisation
yeojohnson = PowerTransformer()
minmax = MinMaxScaler()
sample = df.copy()
sample[:] = minmax.fit_transform(yeojohnson.fit_transform(sample))
simpler = sample.drop(columns=['ptb_rate', 'early_ptb_rate', 'extr_ptb_rate'])

# Reducing dimensionality with PCA
pca = PCA(n_components=0.95)
pca.fit(simpler)
pca_sample = pd.DataFrame(pca.transform(simpler))
pca_sample.index = simpler.index

## SOM Parameters

1. Calculation of ideal SOM map size.

In [13]:
P = sample.shape[0]
M = (5*np.sqrt(P))

print(M)
print(1+int(np.sqrt(M)))

Mh, Mv = 1+int(np.sqrt(M)), 1+int(np.sqrt(M))

371.78622890042607
20


2. Parameters configurations to test for SOM network

In [14]:
import itertools

sigma = list(range(2, 6))
lr = [.1]
neighbourhood = ['gaussian', 'triangle', 'bubble']
activation = ['manhattan', 'euclidean', 'cosine']
base_sample = ['normal', 'pca_reduced']
base_sample_dict = {'normal' : simpler, 'pca_reduced' : pca_sample}

configs = list(itertools.product(neighbourhood, sigma, lr, activation, base_sample))

## SOM Training

Steps for each configuration:
1. Train network
2. Get municipality classifications
3. Cut hierarchical tree using Dynamic Cut
4. Build and save info dataset

In [27]:
classf = {}
firstrun_flag = 0

for c in tqdm(configs):
    input = base_sample_dict[c[4]]

    # Train SOM network
    som = MiniSom(Mh, Mv, input.shape[1], sigma=c[1], learning_rate=c[2], activation_distance=c[3],
                        neighborhood_function=c[0], topology='rectangular')
    som.train_batch(input.to_numpy(), 100000, verbose=False)
    
    # Get results
    winner_coordinates = np.array([som.winner(x) for x in input.to_numpy()]).T
    cluster_index = np.ravel_multi_index(winner_coordinates, (Mh,Mv))
    X = som.get_weights().reshape(Mh*Mv, len(input.columns))
    Z = linkage(X, method='ward',)

    # Results DataFrame
    xdf = pd.DataFrame(X)
    xdf.columns = input.columns
    cindex_neu = xdf.index
    
    # Dynamic Cut
    tdist = c[3] if c[3] != 'manhattan' else 'cityblock'
    distances = pdist(X, tdist)
    clusters = cutreeHybrid(Z, distances, verbose=0, minClusterSize=20)
    transl = dict(zip([i for i in range(0,Z.shape[0]+1)], clusters['labels']))
    cindex = pd.Series(cindex_neu).map(transl).values
    cluster_mun = pd.Series(cluster_index).map(transl).values
    
    ## Map-specific data, used to generate SOM visualisations
    # order = sample.groupby(cluster_mun)[['ptb_rate']].mean().sort_values(by='ptb_rate', ascending=False).index.to_list()
    # marc_ = sample.groupby(cluster_mun)['ptb_rate'].mean()
    # marc = sample.groupby(cluster_mun)['ptb_rate'].mean() > sample.ptb_rate.mean()
    # xx, yy = som.get_euclidean_coordinates()
    # umatrix = som.distance_map()
    # weights = som.get_weights()
    
    # Build general dataset of municipal classification
    resultados = sample.reset_index()[['cd_ibge']]
    resultados['cell']     = cluster_index
    resultados['major']    = cluster_mun
    resultados['neig']     = c[0]
    resultados['sigma']    = c[1]
    resultados['lr']       = c[2]
    resultados['dist']     = c[3]
    resultados['dataset']  = c[4]
    resultados['qe']       = som.quantization_error(input.to_numpy())
    resultados['dbs']      = davies_bouldin_score(sample[~np.isnan(cluster_mun)], cluster_mun[~np.isnan(cluster_mun)])
    
    if firstrun_flag == 0:
        resultados.to_csv(f'{folder_name}/som_cluster_output.csv')
        firstrun_flag += 1
    else:
        resultados.to_csv(f'{folder_name}/som_cluster_output.csv', mode='a', header=False)
        

  0%|                                                                                           | 0/72 [00:00<?, ?it/s]

..cutHeight not given, setting it to 25.184485612183064  ===>  99% of the (truncated) height range in dendro.


  1%|█▏                                                                                 | 1/72 [00:28<34:00, 28.73s/it]

..cutHeight not given, setting it to 26.856397511506497  ===>  99% of the (truncated) height range in dendro.


  3%|██▎                                                                                | 2/72 [00:48<27:20, 23.43s/it]

...

 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [28:03<00:51, 25.81s/it]

..cutHeight not given, setting it to 27.274258880457406  ===>  99% of the (truncated) height range in dendro.


 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [28:38<00:28, 28.39s/it]

..cutHeight not given, setting it to 25.300301126898756  ===>  99% of the (truncated) height range in dendro.


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [29:03<00:00, 24.22s/it]
